# State Pattern Example : Bubblegum Machine

Exemple repris depuis HFDP --> Machine à chewing gum.

## Méthodologie

* Lister les états (No quarter/Has quarter/Out of gumball/Gumball sold)
* Lister les actions possibles dans le système (Insert quarter / Turn crank / Ejects Quarter / Dispense)
* Définir une interface reprenant toutes les actions possibles, puis définir une classe concrète par état.


Ressemble à Stratégie, car on définit les actions possibles dans une interface, et ces actions sont ensuite implémentées différemment en fonction de l'état du système. On va donc avoir une variable "state" représentant l'état actuel du système, et sur laquelle on appelle les différentes actions réalisées.  
La grande différence avec Stratégie est que les actions réalisées ont le pouvoir de changer State et donc de faire en sorte qu'ensuite l'action appelée puisse avoir un sens différent, car l'état du système aura changé.

GumballlMachine : 
* Initialisation une instance de chacun des état
* A une propriété "State" initialisée à l'un de ces états
* A un setter pour State permettant de modifier l'état
* A des getters pour chacun des états

Chaque état :
* A une référence à gumballMachine
* De façon à pouvoir utiliser les getters des différents états pour modifier la valeur de State.

C'est cette gynmastique qui explique des lignes comme la suivante dans les états concrets : 
```python
self._gumballMachine.setState(self._gumballMachine.getHasQuarterState())
```

In [1]:
import random

## State Class

In [2]:
class State:
    
    def insertQuarter():
        pass
    
    def ejectQuarter():
        pass
    
    def turnCrank():
        pass
    
    def dispense():
        pass

## Gumball Machine

In [3]:
class GumballMachine:
    
    soldOutState : State = None
    noQuarterState : State = None
    hasQuarterState : State = None
    soldState : State = None
    winnerState : State = None
    
    state : State = soldOutState
    count: int = 0
        
    def __init__(self, numberGumballs: int):
        self.soldOutState = SoldOutState(self)
        self.noQuarterState = NoQuarterState(self)
        self.hasQuarterState = HasQuarterState(self)
        self.soldState = SoldState(self)
        self.winnerState = WinnerState(self)
        self.count = numberGumballs
        
        if self.count > 0 :
            self.state = self.noQuarterState
    
    def getSoldOutState(self)->State:
        return self.soldOutState
    
    def getNoQuarterState(self)->State:
        return self.noQuarterState
    
    def getHasQuarterState(self)->State:
        return self.hasQuarterState
    
    def getSoldState(self)->State:
        return self.soldState
    
    def getWinnerState(self)->State:
        return self.winnerState
    
    def getCount(self) -> int:
        return self.count
    
    def insertQuarter(self) -> None:
        self.state.insertQuarter()
        
    def ejectQuarter(self) -> None:
        self.state.ejectQuarter()
        
    def turnCrank(self) -> None:
        self.state.turnCrank()
        self.state.dispense()
    
    def setState(self, state: State):
        self.state = state
        
    def releaseBall(self) -> None:
        print('A gumball comes rolling out the slot...')
        if self.count != 0 :
            self.count = self.count - 1
    
        
    

## Concrete States

In [4]:
class SoldState(State):
    
    _gumballMachine: GumballMachine = None
    
    def __init__(self, gumballMachine: GumballMachine):
        self._gumballMachine = gumballMachine
    
    def insertQuarter(self):
        print("You inserted a quarter")
        self._gumballMachine.setState(self._gumballMachine.getHasQuarterState())
    
    def ejectQuarter(self):
        print("Sorry, you already turned the crank")
    
    def turnCrank(self):
        print("Sorry, turning twice doesn't get you another gumball")
    
    def dispense(self):
        self._gumballMachine.releaseBall()
        if self._gumballMachine.getCount() >0 :
            self._gumballMachine.setState(self._gumballMachine.getNoQuarterState())
        else:
            print("Ooops out of gumballs!")
            self._gumballMachine.setState(self._gumballMachine.getSoldOutState())

In [5]:
class WinnerState(State):
    
    _gumballMachine: GumballMachine = None
        
    def __init__(self, gumballMachine: GumballMachine):
        self._gumballMachine = gumballMachine
    
    def insertQuarter(self):
        print("You inserted a quarter")
        self._gumballMachine.setState(self._gumballMachine.getHasQuarterState())
    
    def ejectQuarter(self):
        print("Sorry, you already turned the crank")
    
    def turnCrank(self):
        print("Sorry, turning twice doesn't get you another gumball")
    
    def dispense(self):
        print("YOU'RE A WINNER!! You get two gumballs for your quarter")
        self._gumballMachine.releaseBall()
        if self._gumballMachine.getCount() == 0 :
            self._gumballMachine.setState(self._gumballMachine.getSoldOutState())
        else:
            self._gumballMachine.releaseBall()
            if self._gumballMachine.getCount() > 0 :
                self._gumballMachine.setState(self._gumballMachine.getNoQuarterState())
            else:
                print("Ooops, out of gumballs")
                self._gumballMachine.setState(self._gumballMachine.getSoldOutState())

In [6]:
class HasQuarterState(State):
    
    _gumballMachine: GumballMachine = None
        
    def __init__(self, gumballMachine: GumballMachine):
        self._gumballMachine = gumballMachine
    
    def insertQuarter(self):
        print("You can't insert another quarter")
    
    def ejectQuarter(self):
        print("Quarter returned")
        self._gumballMachine.setState(self._gumballMachine.getNoQuarterState())
    
    def turnCrank(self):
        print("You turned...")
        winner = random.randrange(10)
        if (winner == 0) and (self._gumballMachine.getCount() > 1):
            self._gumballMachine.setState(self._gumballMachine.getWinnerState())
        else:
            self._gumballMachine.setState(self._gumballMachine.getSoldState())
        
    def dispense(self):
        print("No gumball dispensed")

In [7]:
class SoldOutState(State):
    
    _gumballMachine: GumballMachine = None
        
    def __init__(self, gumballMachine: GumballMachine):
        self._gumballMachine = gumballMachine
    
    def insertQuarter(self):
        print("You can't insert a quarter, the machine is sold out")
    
    def ejectQuarter(self):
        print("You can't eject a quarter, you haven't inserted a quarter yet")
    
    def turnCrank(self):
        print("You turned, but there is no gumball")
    
    def dispense(self):
        print("No gumball dispensed")

In [8]:
class NoQuarterState(State):
    
    _gumballMachine: GumballMachine = None
        
    def __init__(self, gumballMachine: GumballMachine):
        self._gumballMachine = gumballMachine
    
    def insertQuarter(self):
        print("A quarter was inserted")
        self._gumballMachine.setState(self._gumballMachine.getHasQuarterState())
    
    def ejectQuarter(self):
        print("You haven't inserted a quarter")
    
    def turnCrank(self):
        print("You turned but there is no quarter")
    
    def dispense(self):
        print("You need to pay first")

## Classe cliente

In [13]:
class GumballMachineTestDrive:
    
    if __name__ == "__main__" :
        
        gM = GumballMachine(5)
        gM.insertQuarter()
        gM.turnCrank()
        
        gM.insertQuarter()
        gM.ejectQuarter()
        gM.ejectQuarter()
        
        gM.insertQuarter()
        gM.insertQuarter()
        gM.turnCrank()
        gM.turnCrank()
        
        gM.turnCrank()
        gM.insertQuarter()
        gM.turnCrank()
        
        gM.insertQuarter()
        gM.turnCrank()
        gM.insertQuarter()
        gM.turnCrank()
        gM.insertQuarter()
        gM.turnCrank()
        gM.insertQuarter()
        gM.turnCrank()
        gM.insertQuarter()
        gM.turnCrank()
        

A quarter was inserted
You turned...
YOU'RE A WINNER!! You get two gumballs for your quarter
A gumball comes rolling out the slot...
A gumball comes rolling out the slot...
A quarter was inserted
Quarter returned
You haven't inserted a quarter
A quarter was inserted
You can't insert another quarter
You turned...
A gumball comes rolling out the slot...
You turned but there is no quarter
You need to pay first
You turned but there is no quarter
You need to pay first
A quarter was inserted
You turned...
A gumball comes rolling out the slot...
A quarter was inserted
You turned...
A gumball comes rolling out the slot...
Ooops out of gumballs!
You can't insert a quarter, the machine is sold out
You turned, but there is no gumball
No gumball dispensed
You can't insert a quarter, the machine is sold out
You turned, but there is no gumball
No gumball dispensed
You can't insert a quarter, the machine is sold out
You turned, but there is no gumball
No gumball dispensed
You can't insert a quarter, 